# Clustering Individual Household Electric Power Consumption and Future Consumption Regression Analysis.

Our group proposes to use the Individual household electric power consumption data set to look for power consumption trends over time. We plan on clustering the data using descriptive methods to discover patterns and trends. Applying predictive methods such as regression we plan to predict future power consumption.

Dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt
from datetime import datetime
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d

# Preprocessing

## Process and clean the data
Process the data by reading each line, removing the column header information and stripping the semicolon seperators. Then convert the date and time stamps to numeric values and merge the two to have a dataset with all numeric values.

In [ ]:
def time_to_ratio(time_stamp):
    time = datetime.strptime(time_stamp, '%d/%m/%Y %H:%M:%S')
    start = datetime(year=time.year, month=1, day=1)
    end = datetime(year=time.year+1, month=1, day=1)
    return (time - start).total_seconds()/(end - start).total_seconds()

def minutes_from_start(time_stamp, start_stamp):
    time = datetime.strptime(time_stamp, '%d/%m/%Y %H:%M:%S')
    start = datetime.strptime(start_stamp, '%d/%m/%Y %H:%M:%S')
    return (time - start).total_seconds()/60.0

In [ ]:
# read data from text document
with open('household_power_consumption.txt', 'r', encoding='utf-8') as f:
    lines = [line.rstrip('\n') for line in f]

# Remove the '?' uncaptured data if detected
data_raw_reduced = [line for line in lines if '?' not in line] 

# strip the header information and remove semicolons     
data_raw = [l.split(';') for l in data_raw_reduced][1::]

# Convert date and time to a numeric value/ratio
time_ratios = [time_to_ratio(f'{t[0]} {t[1]}') for t in data_raw]

# merge time with raw data removing time stamp strings and replacing with ratios
data_time_raw = [[t, float(gap), float(grp), float(v), float(gi), float(s1), float(s2), float(s3)] for (_, _, gap, grp, v, gi, s1, s2, s3), (t) in zip(data_raw, time_ratios)]


In [ ]:
# Verify columns/rows/data are as expected.
print("Number of rows: {}".format(len(data_time_raw)))
print("Number of columns: {}".format(len(data_time_raw[0])))
print(data_time_raw[:5])

# Convert to np array for better processing.
data_time_np = np.array(data_time_raw, dtype=float)
print("Number of rows: {}".format(data_time_np.shape[0]))
print("Number of columns: {}".format(data_time_np.shape[1]))
print(data_time_np[:5])


In [ ]:
start_time = f'{data_raw[0][0]} {data_raw[0][1]}' # the first time stamp
time_from_start = [minutes_from_start(f'{t[0]} {t[1]}', start_time) for t in data_raw] # array of minutes from first time stamp

## Data Information

In [ ]:
df_data = pd.DataFrame(data_time_np)
df_data
df_data_2006_time = df_data[0][0:21992] 
df_data_2006_ap = df_data[1][0:21992]
df_data_2007_time = df_data[0][21992:543661] 
df_data_2007_ap = df_data[1][21992:543661]
df_data_2008_time = df_data[0][543661:1070566] 
df_data_2008_ap = df_data[1][543661:1070566] 
df_data_2009_time = df_data[0][1070566:1591886]
df_data_2009_ap = df_data[1][1070566:1591886]
df_data_2010_time = df_data[0][1591886:]
df_data_2010_ap = df_data[1][1591886:]

fig_combined=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')

plt.plot(df_data_2006_time, df_data_2006_ap,'r--',
        df_data_2007_time, df_data_2007_ap, 'b--',
        df_data_2008_time, df_data_2008_ap, 'g--',
        df_data_2009_time, df_data_2009_ap, 'c--',
        df_data_2010_time, df_data_2010_ap, 'y--', )
plt.xlabel('Date of Year')
plt.ylabel('Active Power (kW)')

In [ ]:
df_data.describe()

In [ ]:
fig_separate=plt.figure(figsize=(10,10), dpi= 100, facecolor='w', edgecolor='k')
ax1 = plt.subplot(511)
plt.plot(df_data_2006_time, df_data_2006_ap, 'r--')
ax2 = plt.subplot(512,sharex=ax1)
plt.plot(df_data_2007_time, df_data_2007_ap, 'b--')
ax2 = plt.subplot(513,sharex=ax1,ylabel='Active Power(kW)')
plt.plot(df_data_2008_time, df_data_2008_ap, 'g--')
ax3 = plt.subplot(514,sharex=ax1)
plt.plot(df_data_2009_time, df_data_2009_ap, 'c--')
ax4 = plt.subplot(515,sharex=ax1,xlabel='Date of Year')
plt.plot(df_data_2010_time, df_data_2010_ap, 'y--')

## Normalization

In [ ]:
## Additional Preprocessing Steps here ##
global_power = data_time_np[:,1].copy() # global power - to check trends

# Normalize by max value in a column
for i in range(1, data_time_np.shape[1]):
    data_time_np[:,i] *= (1.0/data_time_np[:,i].max())


In [ ]:
plt.figure(1)
plt.plot(time_from_start, global_power)
plt.figure(2)
plt.plot(time_from_start, data_time_np[:,1])

print(data_time_np[:5])

## Dimensionality Reduction

Applying different Dimensionality reduction techniques, first to get a general visualization of the data, and second choosing a component factor that captures most of the variance.

In [ ]:
# Reduced the dimensionality to be able to plot in 2D
pca = PCA(n_components=2, svd_solver='full')
data_time_np_vis = pca.fit_transform(data_time_np)

# Verify Reduction
print(data_time_np_vis.shape)

# Visualize the Data
plt.scatter(data_time_np_vis[:, 0], data_time_np_vis[:, 1], s=1)
plt.axis('off')
plt.show()

plt.rcParams["figure.figsize"] = (20, 20)

In [ ]:
#Selecting proper n components for PCA based on explained variance 
pca_n = PCA().fit(data_time_np)
plt.rcParams["figure.figsize"] = (9,5)

fig, ax = plt.subplots()
xi = np.arange(1, 9, step=1)
y = np.cumsum(pca_n.explained_variance_ratio_)
plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 11, step=1)) 
plt.ylabel('Cumulative Variance')
plt.title('PCA')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% Cut-Off ', color = 'red', fontsize=12)

ax.grid(axis='x')
plt.show()

In [ ]:
pca = PCA(n_components=4)
data_time_np_reduced = pca.fit_transform(data_time_np)

print(data_time_np_reduced.shape)

In [ ]:
print(f'Explained Variance from 4 components: {pca.explained_variance_ratio_.sum()*100:.2f}%')

In [ ]:
#Selecting proper n components for SVD based on explained variance 
svd_n = TruncatedSVD(n_components = 7).fit(data_time_np)
plt.rcParams["figure.figsize"] = (9,5)

fig, ax = plt.subplots()
xi = np.arange(1, 8, step=1)
y = np.cumsum(svd_n.explained_variance_ratio_)
plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 8, step=1)) 
plt.ylabel('Cumulative Variance')
plt.title('SVD')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% Cut-Off ', color = 'red', fontsize=12)

ax.grid(axis='x')
plt.show()

In [ ]:
svd = TruncatedSVD(n_components=5)
data_time_np_reduced_svd = svd.fit_transform(data_time_np)

print(data_time_np_reduced_svd.shape)

In [ ]:
print(f'Explained Variance from 5 components: {svd.explained_variance_ratio_.sum()*100:.2f}%')

In [ ]:
fig, axs = plt.subplots(8, figsize=(15,30))
for i in range(8):
    axs[i].plot(time_from_start, data_time_np[:,i])
    axs[i].set_title(f'feature {i}')

In [ ]:
data_time_np_reduced = np.delete(data_time_np, [1,2,3,4], axis=1)

fig, axs = plt.subplots(4, figsize=(15,30))
for i in range(4):
    axs[i].plot(time_from_start, data_time_np_reduced[:,i])
    axs[i].set_title(f'feature {i}')

# Cluster Analysis

To get a better representation of the data we will cluster the reduced data using PCA and try a selected approach to cluster against the three metering values and graphing the data in 3D

## KMeans

In [ ]:
def getSSE(cluster):
    '''
    Retrieved the SSE of the passed in cluster
    '''
    kmeans = KMeans(
    init="k-means++",
    n_clusters=1,
    n_init=10,
    max_iter=300,
    random_state=None
    )
    kmeans.fit(cluster)
    return kmeans.inertia_

def biSectingKmeans(cluster, k=7, debug=False):
    '''
    Bisecting K-meanms Algorithmn 
    '''
    
    rows = cluster.shape[0]
    c = []
    cluster_list = []
    for i in range(0, rows):
        c.append(i)
    
    cluster_list.append(c)
  
    for x in range(k-1):
        SEE = 0
        index = 0
        item = None
        for lists in cluster_list:
            x = getSSE(cluster[lists,:])
            if debug:
                print(x)
            if (x > SEE):
                SEE = x
                item = index
            index += 1
        next_cluster = cluster_list.pop(item)
    
        kmeans = KMeans(
        init="k-means++",
        n_clusters=2,
        n_init=10,
        max_iter=300,
        random_state=None)

        kmeans.fit(cluster[next_cluster,:])

        c2 = []
        c3 = []
        for i in range(0, len(kmeans.labels_)):
            if kmeans.labels_[i] == 0:
                c2.append(next_cluster[i])
            elif kmeans.labels_[i] == 1:
                c3.append(next_cluster[i])
            else:
                print("ERROR BREAKING UP CLUSTER LIST!!!")
        cluster_list.append(c2)
        cluster_list.append(c3)

        if debug:
            print("\n")
            print(c2[:20])
            print("\n")
            print(c3[:20])
            print("*************************")

    return cluster_list

def writeClusterFile(cluster, cluster_groups, file_name='Output.dat'):
    rows = cluster.shape[0]
    c = []
    for i in range(0, rows):
        c.append(-1)

    num = 1
    for lists in cluster_groups:
        length = len(lists)
        for i in range(length):
            c[lists[i]] = num
        num += 1

    f = open(file_name, 'w')
    for z in c:
        f.write(str(z))
        f.write('\n')

    f.close()

def printSSE(cluster, cluster_groups):
    cs_see = []

    for lists in cluster_groups:
        cs_see.append(getSSE(cluster[lists,:]))

    cs_see.sort(reverse=True)
    for lists in cs_see:
        print(lists)

    plt.plot(range(1, len(cs_see)+1), cs_see, 'r-o')
    plt.xlabel("Clusters")
    plt.ylabel("SEE")
    plt.show()

In [ ]:
# Run bi-secting K-Means on cluster
cluster_groups = biSectingKmeans(data_time_np_reduced, k=4, debug=True)
print(len(cluster_groups))

In [ ]:
writeClusterFile(data_time_np_reduced, cluster_groups)
printSSE(data_time_np_reduced, cluster_groups)

In [ ]:
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(data_time_np[:, -3], data_time_np[:, -2], data_time_np[:, -1], color = "green")
plt.title("Utility Cluster Group")
 
# show plot
plt.show()

In [ ]:
data_time_np_selected = data_time_np[:,5:8]
print(data_time_np_selected.shape)

In [ ]:
# Run bi-secting K-Means on cluster
cluster_groups_selected = biSectingKmeans(data_time_np_selected, k=6, debug=True)
print(len(cluster_groups_selected))

In [ ]:
writeClusterFile(data_time_np_selected, cluster_groups_selected, file_name='Output_selected.dat')
printSSE(data_time_np_selected, cluster_groups_selected)

In [ ]:
# Creating figure
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Creating plot
for l in cluster_groups_selected: 
    ax.scatter3D(data_time_np_selected[l, -3], data_time_np_selected[l, -2], data_time_np_selected[l, -1])
    plt.title("Meter Cluster Group")

#ax.view_init(0, 0)
 
# show plot
plt.show()

## OPTICS
This clustering process was done in a separate notebook (clustering.ipynb)

In [ ]:
# reduce data to X for continued analysis
X = data_time_np.copy()[:,[5,6,7]]
print(X[:5])

In [ ]:
X_red = X[::5].copy()

X_red = X_red[~np.all(X_red == 0.0, axis=1)]
print(X_red.shape[0])
X_train, X_test = train_test_split(X_red, train_size=0.75, random_state=42)
print(X_train.shape[0])

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter3D(X_train[:,0], X_train[:,1], X_train[:,2])

plt.show()

In [ ]:
# load the labels
labels = np.load('minsamp200maxeps0.5minclust0.01eps0.05.npy')
print(labels.size)

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

for l in set(labels):
    X_bylabel = X_train[labels == l]
    ax.scatter3D(X_bylabel[:,0], X_bylabel[:,1], X_bylabel[:,2])

plt.show()

# Cluster Association with Time